In [1]:
import numpy as np
import random

In [20]:
!pip install pyspark

Defaulting to user installation because normal site-packages is not writeable


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession\
    .builder\
    .appName("recommender")\
    .getOrCreate()
sc = spark.sparkContext

22/12/11 08:42:23 WARN Utils: Your hostname, noing-Latitude-E5550 resolves to a loopback address: 127.0.1.1; using 192.168.1.16 instead (on interface wlp2s0)
22/12/11 08:42:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/11 08:42:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/12/11 08:42:26 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
data = sc.textFile("ml-100k/u.data")
header = data.first() #extract header
data = data.filter(lambda row: row != header)
M = data.map(
  lambda l: l.split('\t')
).map(
  lambda l: (int(l[0]), int(l[1]), float(l[2]))
)


In [4]:
numWorkers = sc.defaultParallelism
numWorkers

4

In [24]:
M.max(lambda x: x[0])

(943, 58, 4.0)

In [5]:
def assignBlockIndex (index, numData, numWorkers):
    blockSize = numData/numWorkers
    if(numData % numWorkers != 0): blockSize = blockSize + 1
    return int(np.floor(index/np.ceil(blockSize)))+1


In [6]:
numFactors = 10
numRows = M.max(lambda x : x[0])[0] + 1
numCols = M.max(lambda x : x[1])[1] + 1
avgRating = M.map(lambda x: x[2]).mean()

#compute the scaling factor that the randomly initialized W and H matrices need to be scaled by so that dot(W_0,H_0) results in values that are similar to the average ratings
scaleRating = np.sqrt(avgRating / numFactors)

tau = 100


In [7]:
W = M.map(lambda x: tuple([int(x[0]),1])).reduceByKey(lambda x,y : x+y).map(lambda x: tuple([x[0], tuple([x[1], np.random.rand(1,numFactors).astype('float16')])])).persist()
H = M.map(lambda x: tuple([int(x[1]),1])).reduceByKey(lambda x,y : x+y).map(lambda x: tuple([x[0], tuple([x[1], np.random.rand(1,numFactors).astype('float16')])])).persist()

Iterater turn

In [28]:
it = 1
beta = 0.1
mse = sc.accumulator(0.0)
nUpdates = sc.accumulator(0.0)
#broadcast the stepsize
#stepSize = sc.broadcast(np.power(tau + it, -beta))
stepSize = sc.broadcast(0.5)
#generate random strata
lam = sc.broadcast(0.1)

perms = np.random.permutation(numWorkers)+1
perms

array([2, 4, 3, 1])

In [8]:
Mblocked = M.keyBy(lambda x: assignBlockIndex(x[0], numRows, numWorkers)).partitionBy(numWorkers)

In [9]:
def SGD(keyed_iterable, stepSize, numFactors,lam, mse, nUpdates, scaleRating):
    iterlist = (next(keyed_iterable))
    key = iterlist[0]
    Miter = iterlist[1][0]
    Hiter = iterlist[1][1]
    Witer = iterlist[1][2]
    
    Wdict = {}
    Hdict = {}
    
    Wout = {}
    Hout = {}
    
    #iterate through H and W and create dictionary of elements
    for h in Hiter:
        Hdict[h[0]] = h[1]
    
    for w in Witer:
        Wdict[w[0]] = w[1]
    #iterate through entries of M and compute L2-loss
    counter = 1
    for m in Miter:
        (i,j,rat) = m
        if i not in Wdict:
            Wdict[i] = tuple([i,np.random.uniform(0,1,numFactors).astype('float32')])
        if j not in Hdict:
            Hdict[j] = tuple([j,np.random.uniform(0,1,numFactors).astype('float32')])

        (Nw, Wprev) = Wdict[i]
        (Nh, Hprev) = Hdict[j]
        error = (rat - np.dot(Wprev, Hprev.T))
        mse += pow((error[0][0]), 2)
        
        Wnew = Wprev - stepSize.value*(-2*error*Hprev+ (2.0*lam.value)*Wprev)
        Hnew = Hprev - stepSize.value*(-2*error*Wprev + (2.0*lam.value)*Hprev)
        nUpdates += 1
        Wdict[i] = tuple([Nw, Wnew])
        Hdict[j] = tuple([Nh, Hnew])
        
    return (tuple(['W',Wdict.items()]), tuple(['H',Hdict.items()]))

In [10]:
#mse = sc.accumulator(0.0)
#nUpdates = sc.accumulator(0)
stepSize = 0.01
max_iter = 100
stepSize = sc.broadcast(stepSize)
for it in range(max_iter):
    mse = sc.accumulator(0.0)
    nUpdates = sc.accumulator(0)

    beta = 0.1
    #broadcast the stepsize
    #stepSize = sc.broadcast(1/np.power(tau + it, beta))
    stepSize = sc.broadcast(stepSize.value * 0.9)
    #generate random strata
    lam = sc.broadcast(0.01)

    perms = np.random.permutation(numWorkers)+1
    perms_dict = {i: val for i, val in enumerate(perms)}
    rev_perms=list(i+1 for i in (dict(sorted(perms_dict.items(), key=lambda item: item[1]))).keys())
    Mfilt = Mblocked.filter(lambda x: perms[x[0]-1]==assignBlockIndex(x[1][1],numCols,numWorkers)).persist()
    Hblocked = H.keyBy(lambda x: rev_perms[assignBlockIndex(x[0], numCols, numWorkers)-1])
    Wblocked = W.keyBy(lambda x: assignBlockIndex(x[0], numRows, numWorkers))
    groupRDD = Mfilt.groupWith(Hblocked, Wblocked).partitionBy(numWorkers)
    WH = groupRDD.mapPartitions(lambda x: SGD(x, stepSize, numFactors,lam, mse, nUpdates,scaleRating))
    W.unpersist()
    H.unpersist()
    W = WH.filter(lambda x: x[0]=='W').flatMap(lambda x: x[1]).persist()
    H = WH.filter(lambda x: x[0]=='H').flatMap(lambda x: x[1]).persist()
    Wvec = W.collect()
    Hvec = H.collect()
    print("MSE/update for {}-th iteration is: {}/{} ".format(it, mse.value, nUpdates.value))
    print("RMSE: {}".format((mse.value/nUpdates.value)))


MSE/update for 0-th iteration is: 66573.65112829208/52640 
RMSE: 1.2646970199143632


MSE/update for 1-th iteration is: 51209.94574737549/53266 
RMSE: 0.9614002505796472


MSE/update for 2-th iteration is: 47851.85164308548/47266 
RMSE: 1.0123947793992611


MSE/update for 3-th iteration is: 49459.249098300934/53040 
RMSE: 0.9324896134672122


MSE/update for 4-th iteration is: 49378.94162797928/51484 
RMSE: 0.959112377204166


...

MSE/update for 49-th iteration is: 40518.720698833466/52718 
RMSE: 0.768593662484037


MSE/update for 50-th iteration is: 42111.20859861374/51866 
RMSE: 0.8119231982148949


ERROR:root:KeyboardInterrupt while sending command.==========>    (11 + 1) / 12]
Traceback (most recent call last):
  File "/home/noing/.local/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/noing/.local/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

Exception in thread "serve RDD 850" java.net.SocketTimeoutException: Accept timed out
	at java.net.PlainSocketImpl.socketAccept(Native Method)
	at java.net.AbstractPlainSocketImpl.accept(AbstractPlainSocketImpl.java:409)
	at java.net.ServerSocket.implAccept(ServerSocket.java:560)
	at java.net.ServerSocket.accept(ServerSocket.java:528)
	at org.apache.spark.security.SocketAuthServer$$anon$1.run(SocketAuthServer.scala:64)


In [32]:
M.filter(lambda x: x[0]==15).collect()

[(15, 405, 2.0),
 (15, 749, 1.0),
 (15, 25, 3.0),
 (15, 331, 3.0),
 (15, 222, 3.0),
 (15, 473, 1.0),
 (15, 678, 1.0),
 (15, 932, 1.0),
 (15, 127, 2.0),
 (15, 685, 4.0),
 (15, 20, 3.0),
 (15, 301, 4.0),
 (15, 278, 1.0),
 (15, 620, 4.0),
 (15, 742, 2.0),
 (15, 137, 4.0),
 (15, 696, 2.0),
 (15, 924, 3.0),
 (15, 289, 3.0),
 (15, 508, 2.0),
 (15, 754, 5.0),
 (15, 18, 1.0),
 (15, 286, 2.0),
 (15, 148, 3.0),
 (15, 864, 4.0),
 (15, 244, 2.0),
 (15, 274, 4.0),
 (15, 9, 4.0),
 (15, 307, 1.0),
 (15, 458, 5.0),
 (15, 476, 4.0),
 (15, 471, 4.0),
 (15, 7, 1.0),
 (15, 937, 4.0),
 (15, 929, 1.0),
 (15, 889, 3.0),
 (15, 591, 2.0),
 (15, 1, 1.0),
 (15, 933, 1.0),
 (15, 459, 5.0),
 (15, 411, 2.0),
 (15, 744, 4.0),
 (15, 815, 1.0),
 (15, 300, 4.0),
 (15, 926, 1.0),
 (15, 409, 3.0),
 (15, 308, 5.0),
 (15, 303, 3.0),
 (15, 251, 2.0),
 (15, 936, 5.0),
 (15, 455, 1.0),
 (15, 306, 5.0),
 (15, 928, 1.0),
 (15, 14, 4.0),
 (15, 269, 5.0),
 (15, 13, 1.0),
 (15, 252, 2.0),
 (15, 927, 4.0),
 (15, 255, 5.0),
 (15, 50

In [33]:
W.filter(lambda x: x[0]==186).map(lambda x: x[1][1]).collect()[0]

array([[0.842 , 0.785 , 0.225 , 0.2095, 0.489 , 0.579 , 0.983 , 0.8164,
        0.405 , 0.738 ]], dtype=float16)

In [34]:
w_i = np.array(W.filter(lambda x: x[0]==15).map(lambda x: x[1][1]).collect()[0])
w_j = np.array(W.filter(lambda x: x[0]==222).map(lambda x: x[1][1]).collect()[0])
np.dot(w_i, w_j.T)

array([[2.908]], dtype=float16)